This notebook illustrates how to use `schema.py` to construct a JSON catalog file from a set of parameter estimation samples and other metadata.

In [ ]:
import numpy as np
import pandas as pd
from ccverify import schema

In [ ]:
def make_dummy_parameter_set(pe_set_name, is_preferred=False):
    """Return schema.ParameterSet with dummy parameter set."""
    link = schema.Link(
        url='https://example.com',
        content_type='posterior_samples',
        description='string')

    samples = pd.DataFrame(
        {'mass_1_source': np.random.normal(10, 1, 100),
         'luminosity_distance': np.random.normal(130, 10, 100)})

    parameter_set = schema.ParameterSet.from_samples(
        samples=samples,
        pe_set_name=pe_set_name,
        data_url='https://zenodo.org/',
        waveform_family='IMRPhenomPv3HM',
        links=[link],
        is_preferred=is_preferred
        )
    return parameter_set


def make_dummy_event():
    """Return ``schema.Event`` with dummy event."""
    search_statistics_1 = [schema.ParameterValue('pastro', decimal_places=2, median=0.98),
                           schema.ParameterValue('far', decimal_places=2, median=1.23, unit='1/y'),
                           schema.ParameterValue('network_snr', decimal_places=2, median=9.87)]

    search_statistics_2 = [schema.ParameterValue('pastro', decimal_places=2, median=0.89),
                           schema.ParameterValue('far', decimal_places=2, median=1.0, unit='1/y'),
                           schema.ParameterValue('network_snr', decimal_places=2, median=9.87)]

    search_results = [schema.SearchResult(pipeline_name='pipeline1',
                                          search_statistics=search_statistics_1),
                      schema.SearchResult(pipeline_name='pipeline2',
                                          search_statistics=search_statistics_2)
                     ]
    pe_sets = [make_dummy_parameter_set('pe_set_1', is_preferred=True),
               make_dummy_parameter_set('pe_set_2', is_preferred=False)]
    return schema.Event(event_name='GW123456_123456',
                        gps=123456789.0,
                        detectors=['H1', 'L1', 'V1'],
                        search=search_results,
                        pe_sets=pe_sets,
                        event_description='string')


def make_dummy_catalog():
    """Return ``schema.Catalog`` with a dummy catalog."""
    events = [make_dummy_event()]
    return schema.Catalog(catalog_name='catalog name',
                          release_date='2023-12-31',
                          catalog_description='catalog description',
                          doi='https://doi.org/12345/',
                          events=events)

In [ ]:
catalog = make_dummy_catalog()

In [ ]:
catalog.to_json('dummy_catalog.json')

In [ ]:
!ccverify dummy_catalog.json